In [32]:
#Upload into Jupiter the excel 'product-cat-dataset.csv'

import pandas as pd
import numpy as np
from sklearn.model_selection import train_test_split
from nltk.stem import PorterStemmer
from nltk import ngrams
import string
from sklearn.feature_extraction.text import TfidfTransformer, TfidfVectorizer 
from sklearn.feature_extraction.text import CountVectorizer
import pickle
from sklearn.model_selection import train_test_split
from sklearn.naive_bayes import GaussianNB
from sklearn.tree import DecisionTreeClassifier
from sklearn.metrics import accuracy_score

#import data
df = pd.read_csv('product-cat-dataset.csv')

In [26]:
#Here I make a copy of the df and drop null values
df2 = df.copy()
df2.dropna(inplace=True)

In [27]:
# Counts the occurrences of each unique value in the "Level_2" column and creates a new column with these counts
counts_col2 = df2.groupby("Level_2")["Level_2"].transform(len)

# Creates a boolean mask indicating whether the count of occurrences for each "Level_2" value is greater than 10
mask = (counts_col2 > 10)

# Filters the DataFrame df2 based on the boolean mask, keeping only the rows where the count of occurrences for the corresponding "Level_2" value is greater than 10
df3 = df2[mask]

# Prints the resulting DataFrame df3, which contains only the rows where the count of occurrences for the corresponding "Level_2" value is greater than 10
print(df3)

                                             Description   Level_1 Level_2  \
0      gerb cap help keep littl on head cov warm day ...  09BF5150   C7E19   
1      newborn inf toddl boy hoody jacket oshkosh b g...  2CEC27F1   ADAD6   
2      tut ballet anym leap foxy fash ruffl tul toddl...  09BF5150   C7E19   
3      newborn inf toddl boy hoody jacket oshkosh b g...  2CEC27F1   ADAD6   
4      easy keep feel warm cozy inf toddl girl hoody ...  2CEC27F1   ADAD6   
...                                                  ...       ...     ...   
10644  term 10 issu on year subscriptionyo sav 75 cov...  90A8B052   C719A   
10645  term 12 issu on year subscriptionyo sav 86 cov...  90A8B052   C719A   
10646  term 9 issu on year subscriptionyo sav 64 cov ...  90A8B052   C719A   
10647  term 26 issu on year subscriptionyo sav 54 cov...  90A8B052   C719A   
10648  term 12 issu on year subscriptionyo sav 60 cov...  90A8B052   C719A   

      Level_3  
0        D06E  
1        98CF  
2        D06E  

In [28]:
# Calculates the number of occurrences of each unique value in the "Level_3" column and creates a new column with these counts
counts_col2 = df2.groupby("Level_3")["Level_3"].transform(len)

# Creates a boolean mask indicating whether the count of occurrences for each "Level_3" value is greater than 10
mask = (counts_col2 > 10)

# Filters the DataFrame df2 based on the boolean mask, keeping only the rows where the count of occurrences for the corresponding "Level_3" value is greater than 10
df3 = df2[mask]

# Prints the resulting DataFrame df3, which contains only the rows where the count of occurrences for the corresponding "Level_3" value is greater than 10
print(df3)

                                             Description   Level_1 Level_2  \
0      gerb cap help keep littl on head cov warm day ...  09BF5150   C7E19   
1      newborn inf toddl boy hoody jacket oshkosh b g...  2CEC27F1   ADAD6   
2      tut ballet anym leap foxy fash ruffl tul toddl...  09BF5150   C7E19   
3      newborn inf toddl boy hoody jacket oshkosh b g...  2CEC27F1   ADAD6   
4      easy keep feel warm cozy inf toddl girl hoody ...  2CEC27F1   ADAD6   
...                                                  ...       ...     ...   
10644  term 10 issu on year subscriptionyo sav 75 cov...  90A8B052   C719A   
10645  term 12 issu on year subscriptionyo sav 86 cov...  90A8B052   C719A   
10646  term 9 issu on year subscriptionyo sav 64 cov ...  90A8B052   C719A   
10647  term 26 issu on year subscriptionyo sav 54 cov...  90A8B052   C719A   
10648  term 12 issu on year subscriptionyo sav 60 cov...  90A8B052   C719A   

      Level_3  
0        D06E  
1        98CF  
2        D06E  

In [29]:
# Extracts the 'Description' column from DataFrame df3
text = df3["Description"]

# Defines a function named process_text that takes a text input and an integer n
def process_text(text1, n):
    
    # Converts all text to lowercase
    text2 = text1.str.lower()
   
    # Replaces punctuation marks with empty space using regular expressions
    text3 = text2.str.replace(r'[\\,.!_:/+%&-]', '', regex=True)
    
    # Splits the text into individual words
    text4 = text3.str.split(" ")
    
    # Applies the PorterStemmer (optional) to reduce words to their root form
    ps = PorterStemmer()
    text5 = text4.apply(lambda x: [ps.stem(y) for y in x])
    
    # Applies n-gram tokenization to the text and converts it to a list
    tokenized = text5.apply(lambda x: list(ngrams(x, n)))
    
    # Returns the tokenized text
    return tokenized

# Calls the process_text function with the 'Description' column of DataFrame df3 and n=1
result = process_text(df3['Description'], 1)

# Prints the result
print(result)

0        [(gerb,), (cap,), (help,), (keep,), (littl,), ...
1        [(newborn,), (inf,), (toddl,), (boy,), (hoodi,...
2        [(tut,), (ballet,), (anym,), (leap,), (foxi,),...
3        [(newborn,), (inf,), (toddl,), (boy,), (hoodi,...
4        [(easi,), (keep,), (feel,), (warm,), (cozi,), ...
                               ...                        
10644    [(term,), (10,), (issu,), (on,), (year,), (sub...
10645    [(term,), (12,), (issu,), (on,), (year,), (sub...
10646    [(term,), (9,), (issu,), (on,), (year,), (subs...
10647    [(term,), (26,), (issu,), (on,), (year,), (sub...
10648    [(term,), (12,), (issu,), (on,), (year,), (sub...
Name: Description, Length: 10627, dtype: object


In [30]:
# Initialize the CountVectorizer with a custom tokenizer function and set lowercase to False
# CountVectorizer converts a collection of text documents to a matrix of token counts
cv = CountVectorizer(tokenizer=lambda x: x, lowercase=False)

# Initialize the TfidfVectorizer
# TfidfVectorizer converts a collection of raw documents to a matrix of TF-IDF features
tfv = TfidfVectorizer()

# Initialize the TfidfTransformer
# TfidfTransformer transforms a count matrix to a normalized TF or TF-IDF representation
tft = TfidfTransformer(smooth_idf=True)

# Apply the CountVectorizer to transform the result (tokenized text) into a bag of words (bow)
bow = cv.fit_transform(result)

# Apply the TfidfTransformer to convert the bag of words (bow) into TF-IDF representation (tfidf)
tfidf = tft.fit_transform(bow)

# Convert the resulting TF-IDF matrix into a dense DataFrame
# This step is to make it easier to visualize, as the TF-IDF matrix is typically sparse
text_tfidf = pd.DataFrame(tfidf.toarray()).astype(dtype='float32')

C:\Users\HP\anaconda3\Lib\site-packages\sklearn\feature_extraction\text.py:525: UserWarning: The parameter 'token_pattern' will not be used since 'tokenizer' is not None'
  warnings.warn(


In [31]:
# Extracting columns 'Level_1', 'Level_2', and 'Level_3' from the DataFrame df
df3 = df[['Level_1', 'Level_2', 'Level_3']]

# Joining the DataFrame text_tfidf (containing TF-IDF representation of text) with df3 on the index
# This step is not necessary , but it's being done here to combine the dataframes
table = text_tfidf.join(df3)

In [33]:
# Train/Test split
train, test =train_test_split(table, test_size=0.2, random_state=888)

In [34]:
# Extracting the 'Level_1', 'Level_2', and 'Level_3' columns from the DataFrame 'train'
# Converting them to strings using the astype method
# This step is performed to ensure that the values are treated as strings
class1 = train['Level_1'].astype(str)
class2 = train['Level_2'].astype(str)
class3 = train['Level_3'].astype(str)

In [35]:
# Create the model for level 1
tree=DecisionTreeClassifier()

#fit the model on the level 1 clasification using 'Description' col data
datavector=train.iloc[:,0:16252]
tree.fit(datavector, class1)

DecisionTreeClassifier()

In [13]:
# Save the model for level 1
with open('Level1.pk', 'wb') as classes:
    pickle.dump(tree,classes)

In [37]:
# Create and save models for level 2 in for loop

for i in np.unique(class1):
    class1_indices=list(class1[class1==i].index)
    all_train=train.loc[class1_indices,:]
    data_train=all_train.iloc[:,0:16252]
    class2_labels=class2.loc[class1_indices]
    
    tree=DecisionTreeClassifier()
    tree.fit(data_train, class2_labels)
   
    with open("Level2_"+i+".pk", 'wb') as classes:
        pickle.dump(tree,classes)

In [38]:
# Create and save models for level 3

for i in np.unique(class2):
    class2_indices=list(class2[class2==i].index)
    all_train2=train.loc[class2_indices,:]
    data_train2=all_train2.iloc[:,0:16252]
    class3_labels=class3.loc[class2_indices]
    
    tree=DecisionTreeClassifier()
    tree.fit(data_train2, class3_labels)
   
    with open("Level3_"+i+".pk", 'wb') as classes:
        pickle.dump(tree,classes)
    

In [39]:
test.head()

,0,1,2,3,4,5,6,7,8,9,...,16245,16246,16247,16248,16249,16250,16251,Level_1,Level_2,Level_3
9286,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,...,0.0,0.0,0.0,0.0,0.0,0.0,0.0,3E1E0D78,9D9EE,05A0
9646,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,...,0.0,0.0,0.0,0.0,0.0,0.0,0.0,3E1E0D78,9D9EE,05A0
2195,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,...,0.0,0.0,0.0,0.0,0.0,0.0,0.0,09BF5150,C7E19,D06E
4471,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,...,0.0,0.0,0.0,0.0,0.0,0.0,0.0,EFEF723B,CB803,627D
1954,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,...,0.0,0.0,0.0,0.0,0.0,0.0,0.0,69286F45,2D5A3,28A7


In [41]:
# Extracting the 'Level_1', 'Level_2', and 'Level_3' columns from the DataFrame 'test'
# Converting them to strings using the astype method
# This step is performed to ensure that the values are treated as strings
testclass1 = test['Level_1'].astype(str)
testclass2 = test['Level_2'].astype(str)
testclass3 = test['Level_3'].astype(str)

In [42]:
# Selecting the features from the test data to be used for prediction
data_test = test.iloc[:, 0:16252]

# Loading the saved model for predicting Level 1
with open('Level1.pk', 'rb') as tree:
    model = pickle.load(tree)

# Using the loaded model to predict Level 1 for the test data
level1_pred = model.predict(data_test)

# Displaying the predictions for Level 1
level1_pred

array(['3E1E0D78', '3E1E0D78', '09BF5150', ..., 'B092BA29', '96F95EEC',
       '69286F45'], dtype=object)

In [43]:
# Looping through unique values in testclass1 (Level 1 classes)
for i in np.unique(testclass1):
    
    # Loading the saved model for predicting Level 2 based on Level 1 class i
    with open('Level2_'+i+'.pk', 'rb') as tree:
        model = pickle.load(tree)
    
    # Finding the indices of rows in test data belonging to Level 1 class i
    class1_indices = list(testclass1[testclass1 == i].index)
    
    # Selecting all rows from the test data that belong to Level 1 class i
    all_test1 = test.loc[class1_indices, :]
    
    # Selecting features from the test data to be used for prediction
    data_test1 = all_test1.iloc[:, 0:16252]
    
    # Using the loaded model to predict Level 2 for the selected test data
    level2_pred = model.predict(data_test)

# Displaying the predictions for Level 2
level2_pred

array(['CB803', 'CB803', 'CB803', ..., '02FA0', '02FA0', '02FA0'],
      dtype=object)

In [44]:
# Looping through unique values in testclass2 (Level 2 classes)
for i in np.unique(testclass2):
    
    # Loading the saved model for predicting Level 3 based on Level 2 class i
    with open('Level3_'+i+'.pk', 'rb') as tree:
        model = pickle.load(tree)
    
    # Finding the indices of rows in test data belonging to Level 2 class i
    class2_indices = list(testclass2[testclass2 == i].index)
    
    # Selecting all rows from the test data that belong to Level 2 class i
    all_test2 = test.loc[class2_indices, :]
    
    # Selecting features from the test data to be used for prediction
    data_test2 = all_test2.iloc[:, 0:16252]
    
    # Using the loaded model to predict Level 3 for the selected test data
    level3_pred = model.predict(data_test)

# Printing the shape of level3_pred
print(level3_pred.shape)

(2126,)


In [45]:
# Converting predictions to numpy arrays if they're not already
level1_pred = np.array(level1_pred)
level2_pred = np.array(level2_pred)
level3_pred = np.array(level3_pred)

# Creating a DataFrame containing the predictions for each level
results = pd.DataFrame({
    'Level1_Pred': level1_pred,
    'Level2_Pred': level2_pred,
    'Level3_Pred': level3_pred
})

# Displaying the first few rows of the results DataFrame
print(results.head())


  Level1_Pred Level2_Pred Level3_Pred
0    3E1E0D78       CB803        7288
1    3E1E0D78       CB803        7288
2    09BF5150       CB803        7288
3    EFEF723B       CB803        7288
4    69286F45       CB803        7288


## Compute Accuracy on each level 

In [46]:
# Level 1 accuracy
from sklearn.metrics import accuracy_score

# Assuming testclass1, testclass2, testclass3 are the actual labels for the test set
# And level1_pred, level2_pred, level3_pred are the predictions from your models

# Compute accuracy for each level
accuracy_level1 = accuracy_score(testclass1, level1_pred)
accuracy_level2 = accuracy_score(testclass2, level2_pred)
accuracy_level3 = accuracy_score(testclass3, level3_pred)

# Print out the accuracy for each level
print(f"Accuracy for Level 1: {accuracy_level1:.4f}")
print(f"Accuracy for Level 2: {accuracy_level2:.4f}")
print(f"Accuracy for Level 3: {accuracy_level3:.4f}")


Accuracy for Level 1: 0.7954
Accuracy for Level 2: 0.0536
Accuracy for Level 3: 0.0094
